# Practical work on graph-cut optimization (part 2, multilevel)

The objective of this PW is the implementation of the $\alpha$-expansion and $\alpha$-$\beta$ swap approaches for grayscale image denoising.

The PyMaxFlow library is used to compute the graph-cut.

In [ ]:
import math
import random
import numpy as np
#%matplotlib notebook
import matplotlib.pyplot as plt

import platform
import tempfile
import os
from scipy import ndimage as ndi

!pip install PyMaxflow==1.2.13
import maxflow

from skimage import io
from bokeh.plotting import figure, output_file, show
from bokeh.plotting import show as showbokeh
from bokeh.io import output_notebook
output_notebook()

def affiche_pour_colab(im,MINI=None, MAXI=None,titre=''): #special colab, don't look
  def normalise_image_pour_bokeh(X,MINI,MAXI):
    if MAXI==None:
      MAXI = np.max(X)
    if MINI==None:
      MINI = np.min(X)
    imt=np.copy(X.copy())
    imt=(np.clip(imt,MINI,MAXI)/(MAXI-MINI))
    imt[imt<0]=0
    imt[imt>1]=1
    imt*=255
    sortie=np.empty((*imt.shape,4),dtype=np.uint8)
    for k in range(3):
      sortie[:,:,k]=imt
    sortie[:,:,3]=255
    return sortie

  img = im
  img=normalise_image_pour_bokeh(np.flipud(im),MINI,MAXI)
  p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")], y_range=[im.shape[0], 0], x_range=[0, im.shape[1]], title=titre)
  # p.x_range.range_padding = p.y_range.range_padding = 0
  # must give a vector of images
  p.image(image=[im], x=0, y=0, dw=im.shape[1], dh=im.shape[0], palette="Greys256", level="image")
  p.xgrid.visible = False
  p.ygrid.visible = False
  showbokeh(p)

def affiche(im,MINI=0.0, MAXI=None,titre='',printname=False):
  affiche_pour_colab(im,MINI=MINI, MAXI=MAXI,titre=titre) # under google colab many options disappear

def display_segmentation_borders(image, bin):
    imagergb = np.copy(image)
    from skimage.morphology import binary_dilation, disk
    contour = binary_dilation(bin,disk(15))^bin
    imagergb[contour==1,0] = 255
    imagergb[contour==1,1] = 0
    imagergb[contour==1,2] = 0
    return imagergb

## 2 Denoising a grayscale image


In this second part of the PW, we are interested in using Markovian methods to **denoise** images with different regularization potentials.

We are interested in denoising the images *Ibruitee.png* and *Ibruitee2.png* which correspond to the same scene perturbed by two noises of different nature.


We will complete programs that call the algorithm of alpha-expansions or Boykov's alpha-beta swap according to the Kolmogorov technique.


Q1: What are the respective expressions for the data attachment potentials in the case of noise following a Gaussian distribution (equation 1) and a Rayleigh distribution (equation 2)?

\begin{equation}
p(y_p|x_p)=\frac{1}{\sqrt{2\pi}\sigma}\exp\left[-\frac{(y_p-x_p)^2}{2\sigma^2}\right],
\end{equation}

\begin{equation}
p(y_p|x_p)=2\frac{y_p}{x_p^2}\exp\left[-\frac{y_p^2}{x_p^2}\right].
\end{equation}

**Your answer &#x270D;**

A1:
the data attachement term for the case of gaussian distribution $U_{att} = -ln(P(y_p|x_p) = -ln(\sqrt{2\pi σ}) + \frac{(y_p-x_p)^2}{2 σ} $  and we can redefine it as : $U_{att}= (y_p-x_p)^2 $

In the case of the Rayleigh distribution we get : $U_{att} = -ln(2)-ln(y_s)+2ln(x_p)+\frac{y_p^2}{x_p^2}$ we can redfine it as : $U_{att} = -ln(y_s)+2ln(x_p)+\frac{y_p^2}{x_p^2}$

Q2: By studying the histogram of a homogeneous area, indicate which type of noise is present in which image.  



**Your answer &#x270D;**

A2: In the first image we noitice that we have a gaussian noise. In the second image we have a Rayleigh noise

In [ ]:
im_obs=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee.png') # Observed image, noisy
im_obs2=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee2.png') # Observed image, noisy
im_orig=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/IoriginaleBW.png') # Reference binary image, to evaluate the quality of the segmentation

I = im_obs
affiche(im_obs,MINI=0.0, MAXI=255.,titre="Noisy image",printname=True)

In [ ]:
# select a homogeneous area to study the histogram
# we can use the area from 45 to 120 and from 175 to 245
affiche(im_obs[:,:],MINI=0.0, MAXI=255.,titre="Noisy image",printname=True)

plt.figure()
plt.hist(im_obs[45:120,175:245].flatten(),50)
plt.title('Dark area image 1')
plt.show()

plt.figure()
plt.hist(im_obs2[45:120,175:245].flatten(),50)
plt.title('Dark area image 2')
plt.show()


We will compare three *a priori* models: the Potts model $\delta_{x_p=x_q}$, the (discrete) total variation $|x_p-x_q|$, and the (quadratic) Gaussian model $(x_p-x_q)^2$.

Q3: Are they metrics or semi-metrics? What can we deduce about the optimization method to use?

**Your answer &#x270D;**

A3:
In order to know wether a model is (semi) metric or not we need to verify the following properties on  the potentiel expression :

Semi -metric if $\forall \alpha ,\beta \in E^2 :$

- $V_c(\alpha ,\beta) =V_c(\beta , \alpha) \geq 0$

- $V_c(\alpha ,\beta) = 0 ⇔ \alpha = \beta$

Metric if we have additianlly  :     

- $V_c(\alpha,\beta) \leq V_c(\gamma,\beta) + V_c(γ,\beta) $


We conclude :     


-Potts is metric

-Quadratic is semi-metric

-total variation is metric

Q4: What are the differences between the alpha expansion method and the alpha-beta swap method?

**Your answer &#x270D;**

A4:

In the alpha-beta swap we are comparing between two classes at a time. We face each pixel to a choice between keeping a current value or swapping it with the other (alpha or beta). However, in the the alpha expansion we are dividing the set of labels into two disjoint sets : $\alpha$ and $\bar{\alpha} = L∖\alpha$ and we are deciding wether to keep the current pixel label or to give it the value $\alpha$. An other important difference is that the $\alpha - \beta $ swap works on the semi metric régularisation expression but the $\alpha$ expension works only on metrics.


There are other implementation details that differs as well like the importance of the intialization for the case of the $\alpha - \beta $ swap and the graph representation in each case : for the $\alpha - \beta $ swap we represent a subset of the pixels but in the $\alpha $ expension we need to represent all of them

In the following sections we will use the functions aexpansion_grid and abswap_grid which perform the alpha-expansion and alpha-beta swap respectively. These functions take as input two arguments for a number of levels L :
- a tensor of the image size containing in the 3rd dimension the data attachment of each pixel for each considered level (unary term)
- a matrix containing the values of the interaction terms between two levels $l_1$ and $l_2$ (depends on the chosen interaction potential)

### 2.1 Denoising in the Gaussian case (synthetic image)

In [ ]:
from maxflow.fastmin import aexpansion_grid, abswap_grid

# Loading images
im_obs=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee.png').astype('float') # Observed image, noisy
im_obs2=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee2.png').astype('float') # Observed image, noisy
im_orig=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/IoriginaleBW.png').astype('float') # Reference binary image, to evaluate the quality of the segmentation

I = im_obs

I = I*255/I.max()
L = 30
# Generates L gray levels for nearsest prototype labeling
levs = np.arange(0, 255, 255/L)
# Calculate data cost as the absolute difference between the label prototype and the pixel value
D = np.double(np.abs((I.reshape(I.shape+(1,)) - levs.reshape((1,1,-1)))**2))

affiche(I,MINI=0.0, MAXI=255.,titre="Noisy image",printname=True)

# Generate nearest prototype labeling
Id = np.argmin(D,2)
affiche(Id*255/L,MINI=0.0, MAXI=255.,titre="Maximum likelihood denoising",printname=True)
print(D.shape)


Q5: Here, what does the array D correspond to? Explain its dimension.

**Your answer &#x270D;**

A5: The array D contains the data attachement term for the gaussian model for each pixel ( hence the size of the image 323 X 261) and for each gray level (30 : the number of levels)

Complete the programs below, and test each regularization model by determining an appropriate beta value each time.

In [ ]:
# Potts model regularization
# beta values of several hundred
beta_Potts =2000
# definition of the regularization matrix V(i,j) for the Potts model
V_Potts = np.double(beta_Potts*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))>0))
affiche(V_Potts)

# Performs the alpha-expansion based on the data attachment D and the regularization V
labels_Potts = aexpansion_grid(D,V_Potts)
affiche(labels_Potts*255/L,MINI=0.0, MAXI=255.0,titre="Graph Cut Denoising, Potts regularization, beta = " + str(beta_Potts),printname=True)

In [ ]:
# TV model regularization
# beta value of the order of tens
beta_TV = 40
# definition of the regularization matrix V(i,j) for the TV model
V_TV = np.double(beta_TV*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))))
affiche(V_TV)

# Performs the alpha-expansion based on the data attachment D and the regularization V
labels_TV = aexpansion_grid(D,V_TV)
print("Calcul TV terminé")
affiche(labels_TV*255/L,MINI=0.0, MAXI=255.0,titre="Graph Cut Denoising, TV regularization, beta = " + str(beta_TV),printname=True)

For the quadratic regularization model we can no longer use the $\alpha$ expension method since it is not a metric. So for this case we will use the $\alpha - \beta$ swap algorithm that supports the semi-metric models like the quadratic

In [ ]:
# Quadratic model regularization
# beta value of the order of tens
beta_quadratic = 6
# definition of the regularization matrix V(i,j) for the quadratic model
V_quadratic =np.double(beta_quadratic*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1))))**2)
affiche(V_quadratic)

# Performs the alpha-beta swap based on the data attachment D and the regularization V
# The abswap_grid contains a no longer supported syntax of slicing and thus the current implementation will fail. To solve this I made some changes in the
# the maxflow.fastmin.py file and corrected it . In order to use it you need to download that file and put it in the same directory as this notebook
labels_Quadratic = abswap_grid(D,V_quadratic)
affiche(labels_Quadratic*255/L,MINI=0.0, MAXI=255,titre="Graph Cut Denoising, TV regularization, beta = " + str(beta_quadratic),printname=True)

Q6: Which regularization model do you think is best? Give the best regularization parameter visually found and comment on the results you get in each of the three cases.

**Your answer &#x270D;**

A6: For this simple image, the potts model gives us the best results. The best parameters obtained are 2000,40,4.
The potts results gives are quite accurate: It is able to identify connex regions and doesn't mix it with others. This is espacially clear in the case of the background or the big black rectangles. This is not the case for the other two models where we are getting mixed values for the same homogenous regions.

### 2.2 Denoising in the case of speckle noise (synthetic image)

Modify the following cells to fit the model for denoising the im_obs2 image.

Q7: Which modifications are needed? (There are several!)

**Your answer &#x270D;**

A7: The data attachement term has to change since we no longer have the gaussian model.

In [ ]:
from maxflow.fastmin import aexpansion_grid
from numpy import log

# Loading image
im_obs2=io.imread('https://perso.telecom-paristech.fr/tupin/TPGRAPHCUT/OLD/Ibruitee2.png').astype('float') # Observed image, noisy
I = im_obs2
I = I*255/I.max()
affiche(I,MINI=0.0, MAXI=255.,titre="Noisy image 2",printname=True)

L = 30
# Generates L gray levels for nearsest prototype labeling
levs = np.arange(1/L, 255, 255/L)

# Calculate data cost as the absolute difference between the label prototype and the pixel value
D = -log(2*I.reshape(I.shape+(1,))/levs.reshape((1,1,-1))**2)+(I.reshape(I.shape+(1,))**2/levs.reshape((1,1,-1))**2)
print()
Id = np.argmin(D,2)

# Generate nearest prototype labeling
Id = np.argmin(D,2)
affiche(Id/L*255,MINI=0.0, MAXI=255.,titre="Maximum likelihood denoising",printname=True)

In [ ]:
# beta value in the order of tenths of a unit
beta_Potts = 0.5
# definition of the Potts potential matrix
V_Potts = np.double(beta_Potts*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))>0))

# Performs the alpha-expansion based on the data attachment D and the regularization V
labels_Potts = aexpansion_grid(D,V_Potts)
affiche(labels_Potts*255/L,MINI=0.0, MAXI=255,titre="Graph Cut Denoising, Potts regularization, beta = " + str(beta_Potts),printname=True)

In [ ]:
# beta value in hundredths of a unit
beta_TV =0.02
# definition of the regularization matrix for TV
V_TV =np.double(beta_TV*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))))

# Performs the alpha-expansion based on the data attachment D and the regularization V
labels_TV =aexpansion_grid(D,V_TV)
print("TV computation completed")
affiche(labels_TV*255/L,MINI=0.0, MAXI=255,titre="Graph Cut Denoising, TV regularization, beta = " + str(beta_TV),printname=True)

Same, here we use the $\alpha - \beta$ swap algo for the quadratic model

In [ ]:
# beta value in the order of thousandths of a unit
beta_quadratic = 0.001
# definition of the regularization matrix for a quadratic potential
V_quadratic =np.double(beta_quadratic*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1))))**2)

# Performs the alpha-expansion based on the data attachment D and the regularization V
labels_Quadratic = abswap_grid(D,V_quadratic)
affiche(labels_Quadratic*255/L,MINI=0.0, MAXI=255.,titre="Graph Cut Denoising, Quadratic regularization, beta = " + str(beta_quadratic),printname=True)

### 2.3 Denoising a natural image

Apply one of the methods used above to denoise the noisy cameraman image. Justify your choice.

Q8: Comment on the result obtained.

**Your answer &#x270D;**

A8: It is more intressting to use the TV model since this image isn't constant piecewise like the pervious example. In addition the TV model is a metric and thus this will allow us to use the $\alpha$ expansion algorithm which is better than the $\alpha - \beta$ swap

We can say that for the chosen value of beta, the result was average. We succeded in restoring the objects' contours of the image like the man and the camera, however we couldn't restore the the building in the background. This is due to the high level of the noise present in the image (high noise to signal ratio). Thus, such a result was expected from the begining.



In [ ]:
from skimage.color import rgb2gray
import imageio.v2 as imageio

I_cameraman = imageio.imread('https://people.math.sc.edu/Burkardt/data/tif/cameraman.tif')
I_cameraman_bruit = I_cameraman + np.random.normal(0,80,I_cameraman.shape)
I_cameraman_bruit[I_cameraman_bruit<0] = 0
I_cameraman_bruit[I_cameraman_bruit>255] = 255
print(I_cameraman.max())

plt.figure(figsize=(7,7))
plt.imshow(I_cameraman, cmap='gray')
plt.title('Image without noise')
plt.show()

plt.figure(figsize=(7,7))
plt.imshow(I_cameraman_bruit, cmap='gray')
plt.title('Noisy image')
plt.show()

In [ ]:
I = 255*I_cameraman_bruit/I.max()
L = 250
# Generates L gray levels for nearsest prototype labeling
levs =np.arange(0,255,255/L)
# Calculate data cost as the absolute difference between the label prototype and the pixel value
D =np.double(np.abs((I.reshape(I.shape+(1,))- levs.reshape((1,1,-1)))**2))

# choose a regularization model and compute V matrix
Id= np.argmin(D,2)
affiche(Id, MINI=0.0,MAXI=None, titre="Maximum likelihood denoising",printname=True)
beta_TV= 120
V_TV = np.double(beta_TV*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))))

# compute the appropriate optimization
labels_TV=aexpansion_grid(D,V_TV)
# display the regularized image
affiche(labels_TV, MINI=0.0,MAXI=None,titre="Graph Cut Denoising, TV, beta = " + str(beta_TV),printname=True)

### 2.3 SAR Image Denoising

SAR (Synthetic Aperture Radar) imagery is a radar-based remote sensing modality that provides images of the Earth in all light and weather conditions. A major drawback is the high speckle noise that affects them. The following cell loads an amplitude image acquired by the Sentinel 1-A satellite over the city of Des Moines in the USA. To limit the computation time, we will work on a small rectangle from the image provided.

Adapt one of the methods used previously to denoise the image provided. We will assume that the noise follows a Rayleigh distribution.

Q9: Comment on the result obtained.

We can compare the result with a denoising obtained by a more recent method (SAR2SAR), based on a Deep Learning approach (the code to display it is provided below).

**Your answer &#x270D;**

A9:
We have obtained a relatively good result using the TV model for this chosen beta value, taking into account the high quantity of noise. We have obbtained an acceptable denoised image, however, we have lost certain points of the river contour. The results obtained by the SAR2SAR methode are far superior. Additionaly, we have been able to completly denoise the image in a relatively simlar execution time to that of the alpha expansion for this little part of the image.

In [ ]:
try:
    I_SAR = np.load('noisy_DesMoines_dual_1_corrige_1_VV_AMPLITUDE.npy?dl=1')
except:
    !wget 'https://www.dropbox.com/s/7m2dw3irho8dpzj/noisy_DesMoines_dual_1_corrige_1_VV_AMPLITUDE.npy?dl='
    I_SAR = np.load('noisy_DesMoines_dual_1_corrige_1_VV_AMPLITUDE.npy?dl=1')

affiche(I_SAR,0,100,'Full image')
I_SAR = I_SAR[0:300,0:300]
plt.figure(figsize=(7,7))
plt.imshow(I_SAR,vmax = 100, cmap = 'gray') # The display is done by truncating the dynamic
plt.show()

plt.figure()
plt.hist(I_SAR[100:200,100:200].flatten(),200) # Display of the histogram on an almost homogeneous area
plt.show()

In [ ]:
I = I_SAR*255/I_SAR.max()
L = 255
# Generates L gray levels for nearsest prototype labeling
levs = np.arange(1/L,255,255/L)
# Calculate data cost as the neg-log likelihood
D = -2*log(I.reshape(I.shape+(1,))/levs.reshape((1,1,-1))**2)+(I.reshape(I.shape+(1,))**2/levs.reshape((1,1,-1))**2)
print()
affiche(I,MINI=0.0, MAXI=None,titre="Noisy Image",printname=True)

# choose a regularization model and compute V matrix
Id= np.argmin(D,2)
affiche(Id, MINI=0.0,MAXI=None, titre="Maximum likelihood denoising",printname=True)

beta_TV= 0.15
V_TV = np.double(beta_TV*(np.abs( levs.reshape((-1,1)) - levs.reshape((1,-1)))))

# compute the appropriate optimization
labels_TV=aexpansion_grid(D,V_TV)


# display the regularized image
affiche(labels_TV, MINI=0.0,MAXI=None,titre="Graph Cut Denoising, regularisation TV , beta = " + str(beta_TV),printname=True)
plt.figure()
plt.imshow(labels_TV,cmap='gray')
plt.show()

In [ ]:
# Display of the denoised image by the SAR2SAR method

try:
    I_SAR = np.load('denoised_DesMoines_dual_1_corrige_1_VV_AMPLITUDE.npy?dl=1')
except:
    !wget 'https://www.dropbox.com/s/0f6l0qr6teck5bd/denoised_DesMoines_dual_1_corrige_1_VV_AMPLITUDE.npy?dl=1'
    I_SAR = np.load('denoised_DesMoines_dual_1_corrige_1_VV_AMPLITUDE.npy?dl=1')

affiche(I_SAR,0,100,'Full image')
I_SAR = I_SAR[0:300,0:300]
plt.figure(figsize=(7,7))
plt.imshow(I_SAR,vmax = 100, cmap = 'gray') # The display is done by truncating the dynamic
plt.show()

plt.figure()
plt.hist(I_SAR[100:200,100:200].flatten(),200) # Display of the histogram on an almost homogeneous area
plt.show()